# Analyzing KITTI-tracking data and putting it in a table

(And maybe boxplots if sensible.)

Warning: copypasta from the 'StaticDepthAnalysis' notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

%matplotlib inline

from matplotlib import rc
# Enable full LaTeX support in plot text. Requires a full-fledged LaTeX installation
# on your system, accessible via PATH.
rc('text', usetex=True)

plt.rcParams["figure.figsize"] = (16, 6)
matplotlib.rcParams.update({'font.size': 16})

In [77]:
out_dir = '../fig'

def gen_plots(root, part, eval_completeness):
    # if 'eval_completeness' is false, eval accuracy.
    # TODO maybe also compute results for dynamic parts.
    file_pattern = 'k-99999-kitti-tracking-sequence-{sequence_id:04d}--offset-0-'    \
                   'depth-precomputed-{depth}-voxelsize-0.0500-max-depth-m-20.00-'  \
                   '{fusion}-NO-direct-ref-with-fusion-weights-{part}.csv'
    base = os.path.join(root, file_pattern)
    res = {}
    res_completeness = {}
    
    sequences = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    fusions = ['NO-dynamic', 'dynamic-mode']
    metrics = ['input', 'fusion']
    depths = ['dispnet', 'elas']
    
    res = {}
    res_completeness = {}
    
    # Not a proper header. The real one is in the thesis tex, since it's nested
    # and pointless to generate from code.
    for depth in depths:
        for fusion in fusions:
            for metric in metrics:
                if metric == 'input' and fusion == 'NO-dynamic':
                    print('Input & ', end='')
                    
                if metric == 'fusion':
                    print('Fusion-{} &'.format(depth), end='')
    print()
                    
    
    # Yowza, that's a lot of nesting!
    for sequence_id in sequences:
        seq_count = -1
        
        print('{:02d} &'.format(sequence_id), end='')
        for depth in depths:
            best_key = None
            best_score = -1.0
            
            for fusion in fusions:
                fname = base.format(sequence_id=sequence_id, depth=depth, 
                                        fusion=fusion, part=part)
                df = pd.read_csv(fname)
            
                for metric in metrics:
                    key = "{}-{}-{}-{:02d}".format(metric, depth, fusion, sequence_id)

                    # Do not count frames with no pixels in them. This would distort the 
                    # dynamic reconstruction metrics due to frames containing no objects.
                    ok = (df['{}-total-3.00-kitti'.format(metric)] != 0)

                    err = df['{}-error-3.00-kitti'.format(metric)][ok]
                    tot = df['{}-total-3.00-kitti'.format(metric)][ok]
                    mis = df['{}-missing-3.00-kitti'.format(metric)][ok]
                    cor = df['{}-correct-3.00-kitti'.format(metric)][ok]
                    mis_sep = df['{}-missing-separate-3.00-kitti'.format(metric)][ok]

                    acc_perc = cor / (tot - mis)
                    # When evaluating dynamic parts, sometimes we encounter cases with
                    # e.g., very distant cars where tot == mis.
                    acc_perc = acc_perc[~np.isnan(acc_perc)]
                    completeness = 1.0 - (mis_sep / tot)
                    
                    if eval_completeness:
                        res[key] = completeness
                    else:
                        res[key] = acc_perc
                    
                    
                    mean_acc_perc = acc_perc.mean()
                    mean_com_perc = completeness.mean()
                    
                    # The input should be the same in dynamic and non-dynamic mode.
                    if not (metric == 'input' and fusion == 'dynamic-mode'):
                        if eval_completeness:
                            # Compute and display completeness
                            if mean_com_perc > best_score:
                                best_score = mean_com_perc
                                best_key = key
                        else:
                            # Compute and display accuracy
                            if mean_acc_perc > best_score:
                                best_score = mean_acc_perc
                                best_key = key
                    
                    if -1 == seq_count:
                        seq_count = len(df)
                    elif seq_count != len(df):
                        print("Warning: inconsistent lengths for sequence {:04d}".format(sequence_id))
                        print(sequence_id, depth, fusion, metric, len(df))
                  
#             print("BEST KEY: {}".format(best_key))
            for fusion in fusions:
                for metric in metrics:
                    key = "{}-{}-{}-{:02d}".format(metric, depth, fusion, sequence_id)

                    if not (metric == 'input' and fusion == 'dynamic-mode'):
                        if res[key].mean() is np.nan:
                            # No data for the dynamic parts when doing standard fusion!
                            assert(fusion == 'NO-dynamic')
                            continue
                        elif key == best_key:
                            print(r'\textbf{{{:.4f}}}'.format(res[key].mean()), end='')
                        else:
                            print(r'        {:.4f}   '.format(res[key].mean()), end='')
                            
                        if not (metric == 'fusion' and fusion == 'dynamic-mode'):
                            print('& ', end='')
                     
            if depth == depths[0]:
                print('&', end='\n    ')
            
        print(r'\\')
                        

#                 box_colors.append(colors[depth][metric])
#                 columns.append(key)
#                 box_positions.append(box_offset)
    
    
    
    
gen_plots('../csv/tracking-res/', 'static-depth-result', eval_completeness=True)
# print()

# gen_plots('../csv/tracking-res/', 'dynamic-depth-result')

Input & Fusion-dispnet &Fusion-dispnet &Input & Fusion-elas &Fusion-elas &
00 &        0.9847   & \textbf{0.9869}&         0.9773   &
            0.7154   & \textbf{0.9377}&         0.9071   \\
01 &        0.9835   & \textbf{0.9839}&         0.9646   &
            0.7789   & \textbf{0.9627}&         0.9364   \\
02 &        0.9856   & \textbf{0.9896}&         0.9872   &
            0.8360   & \textbf{0.9784}&         0.9743   \\
03 &        0.9761   & \textbf{0.9805}&         0.9742   &
            0.7961   & \textbf{0.9566}&         0.9513   \\
04 &        0.9883   & \textbf{0.9888}&         0.9837   &
            0.8150   & \textbf{0.9736}&         0.9667   \\
05 &        0.9828   & \textbf{0.9857}&         0.9822   &
            0.7887   & \textbf{0.9679}&         0.9631   \\
06 &        0.9886   & \textbf{0.9921}&         0.9902   &
            0.8233   & \textbf{0.9610}&         0.9504   \\
07 &\textbf{0.9882}&         0.9878   &         0.9733   &
            0.7758   & \textbf{0.